In [2]:
import pandas as pd
import numpy as np

In [26]:
df_base = pd.read_csv("C:/Users/Antoine/Documents/AquaVision/donnees/brutes/eau_potable/communes_AEP_2023.csv", encoding="Windows-1252", sep=";", low_memory=False)

In [5]:
df_base

,DPT de la commune,Nom de la commune adhérente,Identifiant SISPEA de la commune adhérente,Code INSEE de la commune adhérente,Population de la commune,Type d'adhésion,Nom de la collectivité via laquelle la commune adhère,N° SIREN de la collectivité via laquelle la commune adhère,Identifiant SISPEA de la collectivité via laquelle la commune adhère,Nom de la collectivité de l'entité de gestion à laquelle la commune adhère,...,Zone tarifaire 1,Tarif zone tarifaire 1,Zone tarifaire 2,Tarif zone tarifaire 2,Zone tarifaire 3,Tarif zone tarifaire 3,Zone tarifaire 4,Tarif zone tarifaire 4,Zone tarifaire 5,Tarif zone tarifaire 5
0,01,Saint-Trivier-de-Courtes,640,01388,1124.0,Adhérent direct,NaN,NaN,NaN,SYNDICAT D'EAU POTABLE SAONE VEYLE REYSSOUZE,...,Indicateur tarif,"2,5",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01,Béréziat,322,01040,490.0,Adhérent direct,NaN,NaN,NaN,SYNDICAT D'EAU POTABLE SAONE VEYLE REYSSOUZE,...,Indicateur tarif,"2,5",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01,Reyssouze,579,01323,1035.0,Adhérent direct,NaN,NaN,NaN,SYNDICAT D'EAU POTABLE SAONE VEYLE REYSSOUZE,...,Indicateur tarif,"2,5",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01,Boissey,331,01050,384.0,Adhérent direct,NaN,NaN,NaN,SYNDICAT D'EAU POTABLE SAONE VEYLE REYSSOUZE,...,Indicateur tarif,"2,5",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01,Sermoyer,652,01402,670.0,Adhérent direct,NaN,NaN,NaN,SYNDICAT D'EAU POTABLE SAONE VEYLE REYSSOUZE,...,Indicateur tarif,"2,5",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33944,92,Courbevoie,36587,92026,82701.0,Adhérent via une collectivité,Paris Ouest La Défense (T4),200057982.0,196812.0,Sénéo,...,Indicateur tarif,"2,45",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33945,92,La Garenne-Colombes,36590,92035,29890.0,Adhérent via une collectivité,Paris Ouest La Défense (T4),200057982.0,196812.0,Sénéo,...,Indicateur tarif,"2,45",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33946,92,Suresnes,36606,92073,49992.0,Adhérent via une collectivité,Paris Ouest La Défense (T4),200057982.0,196812.0,Sénéo,...,Indicateur tarif,"2,45",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33947,92,Gennevilliers,36591,92036,50096.0,Adhérent via une collectivité,Boucle nord de Seine (T5),200057990.0,218377.0,Sénéo,...,Indicateur tarif,"2,45",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# Filtrage du df pour récupérer les communes que l'on veut
df_base = df_base[df_base['DPT de la commune'].isin(['75', '77', '78', '91', '92', '93', '94', '95'])][["DPT de la commune", "Nom de la commune adhérente", "Code INSEE de la commune adhérente", "Identifiant SISPEA de la collectivité de l'entité de gestion à laquelle la commune adhère"]]
df_base = df_base.rename({"Code INSEE de la commune adhérente" : "Code INSEE", "Nom de la commune adhérente" : "Commune", "DPT de la commune" : "Departement"}, axis = 1)

In [28]:
df_base

,Departement,Commune,Code INSEE,Identifiant SISPEA de la collectivité de l'entité de gestion à laquelle la commune adhère
9379,78,Mondreville,78413,175062
9413,78,Dannemarie,78194,174831
15681,77,Bransles,77050,67568
21997,95,Ronquerolles,95529,167404
29162,77,Larchant,77244,152310
...,...,...,...,...
33944,92,Courbevoie,92026,182807
33945,92,La Garenne-Colombes,92035,182807
33946,92,Suresnes,92073,182807
33947,92,Gennevilliers,92036,182807


In [41]:
dico = {"annee" : [], "lignes" : [], "NaN" : []}


for annee in range(2012, 2023):
    dico["annee"].append(annee)
    df = pd.read_csv("C:/Users/Antoine/Documents/AquaVision/donnees/brutes/eau_potable/tarifs_AEP_" + str(annee) + ".csv", encoding="Windows-1252", sep=";", low_memory=False)
    
    # Modification des valeurs de la colonn id SISPEA pour futur merge
    df['Id SISPEA de la collectivité'] = df['Id SISPEA de la collectivité'].str.replace('CollectivityId=', '').astype(int)

    # On ne garde donc que cette colonne et la D102.0:
    df_clean = df[['Id SISPEA de la collectivité', 'D102.0']]

    df_test = pd.merge(df_base, df_clean, how = 'left', left_on = "Identifiant SISPEA de la collectivité de l'entité de gestion à laquelle la commune adhère", right_on = "Id SISPEA de la collectivité").drop(["Identifiant SISPEA de la collectivité de l'entité de gestion à laquelle la commune adhère", "Id SISPEA de la collectivité"], axis = 1)

    dico["NaN"].append(len(df_test[df_test['D102.0'].isna() == True]))
    dico["lignes"].append(len(df_test))

df_final = pd.DataFrame(dico)
df_final["%NaN"] = round(df_final['NaN']/df_final["lignes"] *100 , 2)

df_final

,annee,lignes,NaN,%NaN
0,2012,1411,972,68.89
1,2013,1523,771,50.62
2,2014,1537,749,48.73
3,2015,1481,881,59.49
4,2016,3924,777,19.80
5,2017,5437,1698,31.23
6,2018,5105,1168,22.88
7,2019,8196,2319,28.29
8,2020,10464,865,8.27
9,2021,10707,853,7.97


In [31]:

df_list = []

for annee in range(2012, 2024):
    df = pd.read_csv("C:/Users/Antoine/Documents/AquaVision/donnees/brutes/eau_potable/tarifs_AEP_" + str(annee) + ".csv", encoding="Windows-1252", sep=";", low_memory=False)
    
    df['Id SISPEA de la collectivité'] = df['Id SISPEA de la collectivité'].str.replace('CollectivityId=', '').astype(int)

    df = df[['Id SISPEA de la collectivité', 'D102.0']]    

    df_temp = pd.merge(df_base, df, how = 'left', left_on = "Identifiant SISPEA de la collectivité de l'entité de gestion à laquelle la commune adhère", right_on = "Id SISPEA de la collectivité").drop(["Identifiant SISPEA de la collectivité de l'entité de gestion à laquelle la commune adhère", "Id SISPEA de la collectivité"], axis = 1)

    
    df_temp['D102.0'] = df_temp['D102.0'].str.replace(",", ".").astype(float)

    df_temp = df_temp.fillna(0)

    df_temp = df_temp.groupby('Commune').max()

    df_temp['Année'] = annee

    df_temp = df_temp.reset_index()

    df_list.append(df_temp)

# Concatenation de tous les DataFrames accumulés dans la liste
df_prix = pd.concat(df_list, ignore_index = True)

df_prix['D102.0'] = df_prix['D102.0'].apply(lambda x : np.nan if x ==0 else round(x , 2))

df_prix





,Commune,Departement,Code INSEE,D102.0,Année
0,Abbéville-la-Rivière,91,91001,NaN,2012
1,Ableiges,95,95002,NaN,2012
2,Ablis,78,78003,2.37,2012
3,Ablon-sur-Seine,94,94001,2.05,2012
4,Achères,78,78005,NaN,2012
...,...,...,...,...,...
13999,Étrépilly,77,77173,2.60,2023
14000,Évecquemont,78,78227,3.64,2023
14001,Évry-Courcouronnes,91,91228,3.06,2023
14002,Évry-Grégy-sur-Yerre,77,77175,5.56,2023


In [32]:
df_prix.to_csv("C:/Users/Antoine/Documents/AquaVision/donnees/nettoyees/tarifs_eau_potable.csv", index = False)

In [36]:

df_list = []

for annee in range(2012, 2024):
    df = pd.read_csv("C:/Users/Antoine/Documents/AquaVision/donnees/brutes/assainissement/tarifs_AC_" + str(annee) + ".csv", encoding="Windows-1252", sep=";", low_memory=False)
    
    df['Id SISPEA de la collectivité'] = df['Id SISPEA de la collectivité'].str.replace('CollectivityId=', '').astype(int)

    df = df[['Id SISPEA de la collectivité', 'D204.0']]    

    df_temp = pd.merge(df_base, df, how = 'left', left_on = "Identifiant SISPEA de la collectivité de l'entité de gestion à laquelle la commune adhère", right_on = "Id SISPEA de la collectivité").drop(["Identifiant SISPEA de la collectivité de l'entité de gestion à laquelle la commune adhère", "Id SISPEA de la collectivité"], axis = 1)

    
    df_temp['D204.0'] = df_temp['D204.0'].str.replace(",", ".").astype(float)

    df_temp = df_temp.fillna(0)

    df_temp = df_temp.groupby('Commune').max()

    df_temp['Année'] = annee

    df_temp = df_temp.reset_index()

    df_list.append(df_temp)

# Concatenation de tous les DataFrames accumulés dans la liste
df_prix = pd.concat(df_list, ignore_index = True)

df_prix['D204.0'] = df_prix['D204.0'].apply(lambda x : np.nan if x ==0 else round(x , 2))

df_prix





,Commune,Departement,Code INSEE,D204.0,Année
0,Abbéville-la-Rivière,91,91001,NaN,2012
1,Ableiges,95,95002,NaN,2012
2,Ablis,78,78003,NaN,2012
3,Ablon-sur-Seine,94,94001,NaN,2012
4,Achères,78,78005,NaN,2012
...,...,...,...,...,...
13999,Étrépilly,77,77173,2.3,2023
14000,Évecquemont,78,78227,NaN,2023
14001,Évry-Courcouronnes,91,91228,NaN,2023
14002,Évry-Grégy-sur-Yerre,77,77175,NaN,2023


In [37]:
df_prix.to_csv("C:/Users/Antoine/Documents/AquaVision/donnees/nettoyees/tarifs_assainissement.csv", index = False)

In [6]:
df_prix_A = pd.read_csv("C:/Users/Antoine/Documents/AquaVision/donnees/nettoyees/tarifs_assainissement.csv")
df_prix_EP = pd.read_csv("C:/Users/Antoine/Documents/AquaVision/donnees/nettoyees/tarifs_eau_potable.csv")

In [8]:
df_prix = pd.merge(df_prix_A, df_prix_EP, how = 'outer', on = ['Commune', 'Departement', 'Code INSEE', 'Année'])

In [9]:
df_prix

,Commune,Departement,Code INSEE,D204.0,Année,D102.0
0,Abbéville-la-Rivière,91,91001,NaN,2012,NaN
1,Abbéville-la-Rivière,91,91001,NaN,2013,NaN
2,Abbéville-la-Rivière,91,91001,NaN,2014,NaN
3,Abbéville-la-Rivière,91,91001,NaN,2015,NaN
4,Abbéville-la-Rivière,91,91001,NaN,2016,NaN
...,...,...,...,...,...,...
13999,Ézanville,95,95229,NaN,2019,3.78
14000,Ézanville,95,95229,NaN,2020,3.65
14001,Ézanville,95,95229,NaN,2021,3.80
14002,Ézanville,95,95229,NaN,2022,NaN


In [10]:
df_prix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14004 entries, 0 to 14003
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Commune      14004 non-null  object 
 1   Departement  14004 non-null  int64  
 2   Code INSEE   14004 non-null  int64  
 3   D204.0       3092 non-null   float64
 4   Année        14004 non-null  int64  
 5   D102.0       9418 non-null   float64
dtypes: float64(2), int64(3), object(1)
memory usage: 656.6+ KB


In [11]:
df_prix_A.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14004 entries, 0 to 14003
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Commune      14004 non-null  object 
 1   Departement  14004 non-null  int64  
 2   Code INSEE   14004 non-null  int64  
 3   D204.0       3092 non-null   float64
 4   Année        14004 non-null  int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 547.2+ KB


In [14]:
df_verif = pd.read_csv("C:/Users/Antoine/Documents/AquaVision/donnees/brutes/assainissement/tarifs_AC_2022.csv", encoding="Windows-1252", sep=";", low_memory=False)

In [21]:
df_verif[df_verif['DPT du siège de la coll.'].isin(['075', '077', '078', '091', '092', '093', '094', '095'])][['DPT du siège de la coll.', 'D204.0']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 517 entries, 10192 to 11941
Data columns (total 2 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   DPT du siège de la coll.  517 non-null    object
 1   D204.0                    331 non-null    object
dtypes: object(2)
memory usage: 12.1+ KB


In [22]:
df_prix_EP.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14004 entries, 0 to 14003
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Commune      14004 non-null  object 
 1   Departement  14004 non-null  int64  
 2   Code INSEE   14004 non-null  int64  
 3   D102.0       9418 non-null   float64
 4   Année        14004 non-null  int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 547.2+ KB
